In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print('length of dataset in characters: {}'.format(len(text)))


length of dataset in characters: 1115393


In [5]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


# create a mapping from character to integer 

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: [itos[i] for i in l]

print(encode('hello there'))
print(decode([46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
['h', 'e', 'l', 'l', 'o', ' ', 't', 'h', 'e', 'r', 'e']


In [8]:
import torch

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
# 理解一下输入输出
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is {}, target is {}".format(context, target))

when input is tensor([18]), target is 47
when input is tensor([18, 47]), target is 56
when input is tensor([18, 47, 56]), target is 57
when input is tensor([18, 47, 56, 57]), target is 58
when input is tensor([18, 47, 56, 57, 58]), target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [13]:
torch.manual_seed(509)
batch_size = 4
block_size = 8

def get_batch(split):
    
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    ## 注意，对每一条input sequence, y不是只有下一个token，y同样也是一个sequence
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x,y


xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)

print("inputs:")
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print("when context is {}, target is {}".format(context, target))
    

inputs:
torch.Size([4, 8])
tensor([[40, 56, 53, 61, 57,  0, 40, 53],
        [59, 54, 54, 53, 57, 43, 42,  1],
        [ 1, 46, 43, 39, 42,  8,  0,  0],
        [39, 63,  1, 46, 39, 42,  1, 61]])
inputs:
torch.Size([4, 8])
tensor([[56, 53, 61, 57,  0, 40, 53, 59],
        [54, 54, 53, 57, 43, 42,  1, 63],
        [46, 43, 39, 42,  8,  0,  0, 16],
        [63,  1, 46, 39, 42,  1, 61, 43]])
when context is tensor([40]), target is 56
when context is tensor([40, 56]), target is 53
when context is tensor([40, 56, 53]), target is 61
when context is tensor([40, 56, 53, 61]), target is 57
when context is tensor([40, 56, 53, 61, 57]), target is 0
when context is tensor([40, 56, 53, 61, 57,  0]), target is 40
when context is tensor([40, 56, 53, 61, 57,  0, 40]), target is 53
when context is tensor([40, 56, 53, 61, 57,  0, 40, 53]), target is 59
when context is tensor([59]), target is 54
when context is tensor([59, 54]), target is 54
when context is tensor([59, 54, 54]), target is 53
when context

nn.Embedding将输入的整数序列装换成密集向量表达:
* 第一个参数num_embeddings：词的个数
* 第二个参数embedding_dim：embedding的维度

# Bigram Model

torch中的dim，代表对这个维度的所有元素进行操作
* 比如torch.cat((idx, next_idx), dim = 1)，就是在1维进行concat
* torch.sum(a, 1, keepdim = True)，就是将1维都加起来

In [18]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  #(B, T, C)
        
        
    def forward(self, idx, targets=None):
        # idx和target就是前面的xb,yb,所有都是(B,T)的
        # 经过embedding之后变成：(B,T,C)
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits, loss = self(idx)  # calling self.forward
            logits = logits[:, -1, :]  # (B,C)
            # 生成概率
            probs = F.softmax(logits, dim = -1)  # (B, C)
            # 根据概率sample
            next_idx = torch.multinomial(probs, num_samples=1)  # (B,1)
            idx = torch.cat((idx, next_idx), dim = 1)
        
        return idx 
    
    
bigram_model = BigramLanguageModel(vocab_size)
logits, loss = bigram_model(xb, yb)

print(logits.shape)
print(logits)

print(''.join(decode(bigram_model.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens = 100)[0].tolist())))
    

torch.Size([256, 65])
tensor([[ 0.0282, -0.4387,  1.0534,  ..., -0.3558, -0.1233,  1.1046],
        [ 0.9527,  0.3031, -1.4552,  ..., -0.2864, -0.1493, -1.6743],
        [-1.0090,  0.5156, -0.1046,  ...,  1.4170,  1.4547, -0.4403],
        ...,
        [ 1.1371, -0.4484,  0.5714,  ...,  0.4964, -0.1508, -0.8930],
        [ 0.0282, -0.4387,  1.0534,  ..., -0.3558, -0.1233,  1.1046],
        [ 0.3090,  0.7605,  0.5701,  ...,  0.6129,  1.1609,  0.0365]],
       grad_fn=<ViewBackward0>)

UBJ&BqmtEv&:Scz o&w-kDmtq!Xx:lQghTU!sHgfhZkZ.ZN$Sza!fRizMzqTuoA,Dx?-AQXcgvzWszOqmaLHaBK:$X&:AKAsiw-z


In [22]:
optimizer = torch.optim.AdamW(bigram_model.parameters(), lr = 1e-3)

batch_size = 32

for steps in range(200):
    xb, yb = get_batch('train')
    logits, loss = bigram_model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
    

4.470495223999023


In [23]:
print(''.join(decode(bigram_model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist())))


;qNxyzQNnCOBySy!kwaFMZmCjDmLcbSGNHMoLYK..GIoeoO3'bMpsoDTZgSkAGUdilOLEBevwRh;bXYPevnt-ajvaHBN 'oLqJAFaOr?rG!owaHDkDp,V?lO&OoinWKh.MgS&:?3fBrS
AJevwqb!Fmk:t;fRxDkiEB
;NJrOW$ktw;dMH
gLiwjbgsu.JeHEwukeZINvai3xPdLEhiE mtXPabpZAB,OlOfDi&ci3ArsjPCfCce
;E:gUq
fcko&:!QNXTYTF3E3WwPAjJ,hAIBZgYN fnRJaYkdkBat,TKIzGP3'o?KnvaNLQSb,TX
!o.kuonGPe;JfURu:pDf'U;3PeOntuW$TZiHgHshkByLM
;JVamYXZLBs?UJNu'atQNLR&bwiR
gvw!fiyp:dVaYfbZUl?In
uPgRr?IBQMKiUrs::yqExUKp dwKEY'eaFORhVpXoDgMwlXVaF.q;&?:t
VbOsinw&tTeVjKqDZLsxLx3t


# Self-Attention

In [31]:
a = torch.tril(torch.ones(3,3))
print(a)

a = a / torch.sum(a, 1, keepdim = True)
print(a)

b =  torch.randint(0,10,(3,2)).float()
print(b)

c = a @ b
print(c)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[4., 9.],
        [0., 2.],
        [8., 0.]])
tensor([[4.0000, 9.0000],
        [2.0000, 5.5000],
        [4.0000, 3.6667]])


In [32]:
torch.manual_seed(1004)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)

x.shape

torch.Size([4, 8, 2])

In [33]:
# version 2: matrix multiplication for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim = True)
xbow = wei @ x 
print(xbow)

tensor([[[-1.6492, -0.6707],
         [-1.1352, -1.1039],
         [-0.8041, -0.7152],
         [-0.4538, -0.2215],
         [ 0.0567, -0.3237],
         [-0.0317, -0.0122],
         [ 0.0367, -0.0481],
         [ 0.0413,  0.0769]],

        [[-0.6925,  0.9243],
         [ 0.1176,  1.0500],
         [ 0.0685,  1.1512],
         [ 0.0457,  0.8229],
         [-0.1824,  0.6395],
         [-0.0724,  0.6345],
         [ 0.1508,  0.6462],
         [ 0.1089,  0.7985]],

        [[ 0.3470,  0.9166],
         [-0.3207,  0.8867],
         [ 0.0820,  0.6172],
         [-0.0962,  0.9935],
         [-0.0474,  0.9025],
         [-0.2767,  0.8389],
         [-0.3780,  0.7107],
         [-0.3710,  0.7142]],

        [[ 0.5993,  1.3617],
         [ 1.1844,  0.6525],
         [ 0.6642,  0.3576],
         [ 0.8054,  0.0113],
         [ 0.5579,  0.0457],
         [ 0.4041, -0.2544],
         [ 0.1834, -0.1784],
         [ 0.0139, -0.3001]]])


In [36]:
# version 3: use softmax 
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = 1)
xbow2 = wei @ x 
torch.allclose(xbow2, xbow)


True

In [37]:
# version 4: masked self-attention

torch.manual_seed(1823)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# single head self-attention

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)

wei = q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) -> (B, T, T)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)

v = value(x)
out = wei @ v 
print(out.shape)

torch.Size([4, 8, 16])


# Layer Norm

In [38]:
class LayerNorm1d:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x-xmean) / torch.sqrt(xvar + self.eps) 
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(211)
module = LayerNorm1d(100)
# batch size 32, 100 dimensional vectors
x = torch.randn(32, 100)
x = module(x)
print(x.shape)
    

torch.Size([32, 100])


In [39]:
# mean, std of one feature across all batch inputs
x[:,0].mean(), x[:, 0].std()

(tensor(0.1098), tensor(0.9865))

In [40]:
# mean std of a single input from the batch of its features 
x[0, :].mean(), x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))

# Full code